In [38]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import t, norm
file = 'problem2.csv'
problem2_data = pd.read_csv(file)

In [39]:
#Part1 - 1: OLS
n = problem2_data.shape[0]
X = np.column_stack((np.ones(n), problem2_data['x']))
Y = problem2_data['y'].values

#Use the week2's notes to calculate on matrix
X_transpose = X.T
XTX = X_transpose.dot(X)
XTX_inv = np.linalg.inv(XTX)
XTY = X_transpose.dot(Y)
beta_ols = XTX_inv.dot(XTY)

residuals = Y - X.dot(beta_ols)
s_squared = (residuals.T.dot(residuals)) / (n - X.shape[1])

sigma_squared = s_squared
cov_matrix_beta = sigma_squared * XTX_inv

print("OLS Results:")
print(f"Beta values (OLS): \n{beta_ols}")
print(f"Sigma: {np.sqrt(s_squared)}")

OLS Results:
Beta values (OLS): 
[-0.08738446  0.7752741 ]
Sigma: 1.008813058320225


In [40]:
#Part1 - 1: MLE
def log_likelihood_normal(params, X, y):
    beta = params[:-1]
    sigma = params[-1]
    
    y_pred = X @ beta
    n = len(y)
    
    ll = -0.5 * n * np.log(2 * np.pi * sigma**2) - (0.5 / sigma**2) * np.sum((y - y_pred)**2)
    
    return -ll

X_mle = np.column_stack([np.ones(len(problem2_data['x'])), problem2_data['x']])
y_mle = problem2_data['y']

initial_params = np.append(np.zeros(X_mle.shape[1]), 1)

mle_fit = minimize(log_likelihood_normal, initial_params, args=(X_mle, y_mle), method='BFGS')

beta_mle = mle_fit.x[:-1]
sigma_mle = mle_fit.x[-1]

print("MLE Model:")
print(f"Beta values (MLE): {beta_mle}")
print(f"Sigma: {sigma_mle}")

MLE Model:
Beta values (MLE): [-0.08738448  0.77527408]
Sigma: 1.0037563087311683


In [42]:
#Part2 
#T-distribution
def log_likelihood_t(params, X, y):
    beta = params[:-2]
    df = params[-2]
    sigma = params[-1]
    y_pred = X @ beta
    resid = (y - y_pred) / sigma
    ll = np.sum(t.logpdf(resid, df, scale=sigma))
    return -ll

# AIC and BIC calculation
def calculate_aic(log_likelihood, num_params):
    return 2 * num_params - 2 * log_likelihood

def calculate_bic(log_likelihood, num_params, n):
    return np.log(n) * num_params - 2 * log_likelihood

def main():
    y_mle = problem2_data["y"].values
    X_mle = np.column_stack([np.ones(len(problem2_data['x'])), problem2_data['x']])

    initial_params_t = [0, 1, 10, 1]
    result_t = minimize(log_likelihood_t, initial_params_t, args=(X_mle, y_mle), method="L-BFGS-B")
    log_likelihood_t_val = -result_t.fun
    intercept_hat_t, beta_hat_t, df_hat_t, sigma_hat_t = result_t.x
    print("\nT-distribution MLE Results:")
    print(f"Intercept: {intercept_hat_t.round(4)}, Beta: {beta_hat_t.round(4)}, df: {df_hat_t.round(4)}, Sigma: {sigma_hat_t.round(4)}")

    aic_t = calculate_aic(log_likelihood_t_val, len(initial_params_t))
    bic_t = calculate_bic(log_likelihood_t_val, len(initial_params_t), len(y_mle))
    print(f"AIC for T-distribution model: {round(aic_t, 4)}")
    print(f"BIC for T-distribution model: {round(bic_t, 4)}")

    initial_params_n = [0, 1, 1]
    result_n = minimize(log_likelihood_normal, initial_params_n, args=(X_mle, y_mle), method="L-BFGS-B")
    log_likelihood_n_val = -result_n.fun
    intercept_hat_n, beta_hat_n, sigma_hat_n = result_n.x
    print("\nNormal distribution MLE Results:")
    print(f"Intercept: {intercept_hat_n.round(4)}, Beta: {beta_hat_n.round(4)}, Sigma: {sigma_hat_n.round(4)}")

    aic_n = calculate_aic(log_likelihood_n_val, len(initial_params_n))
    bic_n = calculate_bic(log_likelihood_n_val, len(initial_params_n), len(y_mle))
    print(f"AIC for normal distribution model: {round(aic_n, 4)}")
    print(f"BIC for normal distribution model: {round(bic_n, 4)}")

    if aic_t < aic_n:
        print("\nThe T-distribution model provides a better fit based on AIC.")
    else:
        print("\nThe Normal distribution model provides a better fit based on AIC.")
    
    if bic_t < bic_n:
        print("The T-distribution model provides a better fit based on BIC.")
    else:
        print("The Normal distribution model provides a better fit based on BIC.")

main()


T-distribution MLE Results:
Intercept: -0.0878, Beta: 0.7737, df: 384.4406, Sigma: 1.1906
AIC for T-distribution model: 545.8889
BIC for T-distribution model: 559.0821

Normal distribution MLE Results:
Intercept: -0.0874, Beta: 0.7753, Sigma: 1.0038
AIC for normal distribution model: 575.0751
BIC for normal distribution model: 584.9701

The T-distribution model provides a better fit based on AIC.
The T-distribution model provides a better fit based on BIC.
